In [42]:
import pandas as pd
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import ast
import re
import preprocessor as p 


In [43]:
data_df = pd.read_json("saved_tweets.json", orient='records')

data_df.head(20)

,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
0,1191213266619551744,Jimmy Butler,JimmyButler,2048,together we can do so much https://t.co/2E4yUq...,2019-11-05 17:54:43.105,2019-11-04 04:39:09,12158,[],5361,None,Twitter for iPhone,None,None,None
1,1189404191766302720,Jimmy Butler,JimmyButler,1622,"heat fans thank you for the support tonight, w...",2019-11-05 17:54:43.105,2019-10-30 04:50:32,14424,[],5361,None,Twitter for iPhone,None,None,None
2,1188955535615365121,Jimmy Butler,JimmyButler,243,bringin’ the heat with my dude @IAmJustise in ...,2019-11-05 17:54:43.105,2019-10-28 23:07:44,1289,"[{'text': 'Zaddy', 'indices': [120, 126]}, {'t...",5361,None,Twitter for iPhone,None,None,None
3,1188529034428403712,Jimmy Butler,JimmyButler,5529,family is not an important thing. it’s everyth...,2019-11-05 17:54:43.105,2019-10-27 18:52:58,26907,[],5361,None,Twitter for iPhone,None,None,None
4,1188248564226580480,Jimmy Butler,JimmyButler,2259,good w @MiamiHEAT,2019-11-05 17:54:43.105,2019-10-27 00:18:28,14573,[],5361,"{'attributes': {}, 'contained_within': [], 'co...",Twitter for iPhone,None,None,None
5,1176406463092330496,Jimmy Butler,JimmyButler,643,up and at em,2019-11-05 17:54:43.105,2019-09-24 08:02:12,4374,[],5361,"{'attributes': {}, 'contained_within': [], 'co...",Twitter for iPhone,None,None,None
6,1176287426089762816,Jimmy Butler,JimmyButler,851,FOUR ANTE MERIDIEM. https://t.co/RbZzP66CAt,2019-11-05 17:54:43.105,2019-09-24 00:09:11,4531,[],5361,"{'attributes': {}, 'contained_within': [], 'co...",Twitter for iPhone,None,None,None
7,1171580357281492992,Jimmy Butler,JimmyButler,67,My recent trip to Senegal https://...,2019-11-05 17:54:43.105,2019-09-11 00:24:58,417,[],5361,None,Twitter for iPhone,None,None,None
8,1160690287825608704,Jimmy Butler,JimmyButler,520,realonepointblankperiod @paulpogba https://t.c...,2019-11-05 17:54:43.105,2019-08-11 23:11:43,4356,[],5361,"{'attributes': {}, 'contained_within': [], 'co...",Twitter for iPhone,None,None,None
9,1146275152474324992,Jimmy Butler,JimmyButler,519,Great win brasil,2019-11-05 17:54:43.105,2019-07-03 04:31:07,6348,[],5361,None,Twitter for iPhone,None,None,None


In [44]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3194 entries, 0 to 3193
Data columns (total 15 columns):
tweet_id             3194 non-null int64
name                 3194 non-null object
screen_name          3194 non-null object
retweet_count        3194 non-null int64
text                 3194 non-null object
mined_at             3194 non-null datetime64[ns]
created_at           3194 non-null datetime64[ns]
favourite_count      3194 non-null int64
hashtags             3194 non-null object
status_count         3194 non-null int64
location             4 non-null object
source_device        3194 non-null object
retweet_text         3194 non-null object
quote_text           3194 non-null object
quote_screen_name    3194 non-null object
dtypes: datetime64[ns](2), int64(4), object(9)
memory usage: 399.2+ KB


In [45]:
tweets = data_df['text']

In [46]:
tweets.head()

0    together we can do so much https://t.co/2E4yUq...
1    heat fans thank you for the support tonight, w...
2    bringin’ the heat with my dude @IAmJustise in ...
3    family is not an important thing. it’s everyth...
4                                    good w @MiamiHEAT
Name: text, dtype: object

In [47]:
dict_tweets = {}

for index, value in tweets.iteritems():
    dict_tweets[index] = value


In [48]:
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence, index, dict_tweets):
    l = []
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    l.append(sentence)
    l.append(str(score))
    dict_tweets[index] = l
    return dict_tweets
    
for index in dict_tweets:
    new_dict_tweets = sentiment_analyzer_scores(dict_tweets[index], index, dict_tweets)
    print("---------------------------------------------------------------------")


together we can do so much https://t.co/2E4yUq8NKt {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
---------------------------------------------------------------------
heat fans thank you for the support tonight, we appreciate y’all. another W. {'neg': 0.0, 'neu': 0.559, 'pos': 0.441, 'compound': 0.7845}
---------------------------------------------------------------------
bringin’ the heat with my dude @IAmJustise in my @Theory__ fit. check out part 1 on @PlayersTribune with @neimanmarcus. #Zaddy #NeimanMarcus #ad https://t.co/NnvaphZuj2 {'neg': 0.0, 'neu': 0.894, 'pos': 0.106, 'compound': 0.3612}
---------------------------------------------------------------------
family is not an important thing. it’s everything. {'neg': 0.185, 'neu': 0.815, 'pos': 0.0, 'compound': -0.1511}
---------------------------------------------------------------------
good w @MiamiHEAT----------------------- {'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound': 0.4404}
--------------------------------

@rebeccambrose &lt;&lt; lol a long way away from Tomball {'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'compound': 0.4215}
---------------------------------------------------------------------
I try and beat life.. because I can't cheat death.. {'neg': 0.0, 'neu': 0.738, 'pos': 0.262, 'compound': 0.357}
---------------------------------------------------------------------
@gbrown5 @scoopjardine &lt;&lt; neither one of y'all get me started {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
---------------------------------------------------------------------
You willing to give up everything, to have it all? Hmmm.. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
---------------------------------------------------------------------
RT @LuolDeng9: Good morning y'all. Have a blessed day. &lt;&lt; Morning Lou {'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compound': 0.7783}
---------------------------------------------------------------------
I meant TGIF lol------------------------ {'neg': 0.

RT @Darrion_Tillman: Watchin Marquette Revealed @mr_2eight1 got bigger since then. ok ok. lol &lt;&lt; in my young days g {'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'compound': 0.7351}
---------------------------------------------------------------------
RT @_MAC_10: @mr_2eight1 It is almost your birthday! Happy early 21st! #almostlegal &lt;&lt; yeah right young folk {'neg': 0.136, 'neu': 0.681, 'pos': 0.183, 'compound': 0.2547}
---------------------------------------------------------------------
RT @ptomas3: @Hannahisalegend @mr_2eight1 he is amazing! even more because he plays on bulls, plus i go to marquette to! can i get a RT  ... {'neg': 0.0, 'neu': 0.76, 'pos': 0.24, 'compound': 0.7682}
---------------------------------------------------------------------
RT @Jles9: God Bless you RT @LongLiveDixie: Dear @Jles9 , I sure hope your feeling better you candy ass conference USA playing pussy.. # ... {'neg': 0.099, 'neu': 0.367, 'pos': 0.534, 'compound': 0.9081}
-------------------------

Just because you can't see something. Doesn't mean it isn't there. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
---------------------------------------------------------------------
Lord grant me the strength to change the things I can. The courage to accept the things I can't. The wisdom to know the difference. {'neg': 0.086, 'neu': 0.558, 'pos': 0.356, 'compound': 0.8282}
---------------------------------------------------------------------
The movie soul food. May be one of the greatest movies of all time. {'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'compound': 0.6369}
---------------------------------------------------------------------
@TajGibson22 I'm at the crib. What y'all on over there lol {'neg': 0.0, 'neu': 0.781, 'pos': 0.219, 'compound': 0.4215}
---------------------------------------------------------------------
@TajGibson22 yo Matt Forte. Where you at. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
-----------------------------------------------------------

---------------------------------------------------------------------
So now I have to find a place for lunch. I have a family with me with two children... Where to eat Chicago? Please help!! {'neg': 0.0, 'neu': 0.773, 'pos': 0.227, 'compound': 0.6792}
---------------------------------------------------------------------
Don't understand why there's traffic right now inchicago!!!!! {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
---------------------------------------------------------------------
Np. Joe Nichols. Gimme that girl-------- {'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'compound': 0.34}
---------------------------------------------------------------------
@cynthia_ann91 trust me I've been hearing about it all night lol {'neg': 0.0, 'neu': 0.596, 'pos': 0.404, 'compound': 0.7269}
---------------------------------------------------------------------
@cynthia_ann91 I'm with them right!! Great people!! And you dad looks 21 lol (he says) {'neg': 0.0, 'neu': 0.617, 'pos': 0.38

RT @robin_jay90: @mr_2eight1 Who do you have winning the Super Bowl? &lt;&lt; I'm wearing my texans jersey right now so who you think? {'neg': 0.0, 'neu': 0.733, 'pos': 0.267, 'compound': 0.8253}
---------------------------------------------------------------------
Jersey we out here---------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
---------------------------------------------------------------------
RT @alicekachtou: Totally just saw @mr_2eight1 at chipotle...sweetness &lt;&lt; shoulda said what up {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
---------------------------------------------------------------------
RT @chicity93: I wouldn't say I'm lucky, I'd say I'm blessed. I work hard everyday and try to stay ready for whatever life throws at me. ... {'neg': 0.12, 'neu': 0.675, 'pos': 0.206, 'compound': 0.5673}
---------------------------------------------------------------------
@GrandeGuapo_10 what up cuzzy----------- {'neg': 0.0, 'neu': 1.0, 'pos'

In [67]:
dict_res = {}
for index, item in dict_tweets.items():
    res = item[1].replace("'", "\"")
    res = ast.literal_eval(res) #transformando a string de dict em um dict real
    dict_res[item[0]] = res
print(dict_res)

{'RT @MattBruenig: Just realizing this morning that once the NBA season starts Westbrook will become the best player to ever wear a Rockets j…': {'neg': 0.0, 'neu': 0.84, 'pos': 0.16, 'compound': 0.6369}, "RT @SkySportsNBA: 🏀⭐️⬆️ At #NBACrossover, Inside The NBA's Kenny Smith says even without the injured @KDTrey5, @KyrieIrving and @BrooklynNe…": {'neg': 0.0, 'neu': 0.889, 'pos': 0.111, 'compound': 0.3089}, 'RT @OrlandoMagic: With the fifth overall pick in the 2000 NBA draft the Orlando Magic select, Mike Millerrrrr! #NBARookieWeek https://t.co/…': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'RT @GXATSHXT: Here is what NBA players thought about Rookie Michael Jordan:\n\nNotice how Walt Frazier says: “Michael gotta realize he’s not…': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'RT @wojespn: The NBA Board of Governors passed a stricter package of measures to enforce compliance with tampering and salary cap circumven…': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound